- To rerun any S-LOCAL network you need a minimum of 32GB of RAM/VRAM memory on the device used for training
- Average time to run a 100 epoch on a Nvidia Tesla V100 GPU is 20 minutes
- A faster implementation of the Locally Connected Layer is available in the Keras API : 
- https://keras.io/api/layers/locally_connected_layers/locall_connected2d/

## Imports

In [1]:
INIT_PATH = "."
DOWNLOAD_DATA = False

# To use google colab processing uncomment following lines

#INIT_PATH = "MyDrive/inference_and_learning"
#from google.colab import drive
#MOUNT_PATH = "/content/drive"
#drive.mount(MOUNT_PATH)
#INIT_PATH = MOUNT_PATH + "/" + INIT_PATH
#DOWNLOAD_DATA = True

# To train LOCAL change to True
LOCAL_TRAIN = False 


# To avoid out of memory error, adjust 
NUM_THREADS = 3

In [2]:
import os.path,sys
sys.path.append(INIT_PATH)

import networks as nnets
import importlib
importlib.reload(nnets)
import pandas as pd, torch
from typing import Tuple, List
import torchvision
import torch
import json
from IPython.display import display, HTML
torch.set_default_dtype(torch.float16)

In [3]:
DATA_PATH = "." # Path to data folder
if (DOWNLOAD_DATA):
    torchvision.datasets.CIFAR10(root=DATA_PATH + '/data', download=True)
    torchvision.datasets.CIFAR100(root=DATA_PATH + '/data', download=True)
    torchvision.datasets.SVHN(root=DATA_PATH +'/data', download=True)

    torchvision.datasets.CIFAR10(root=DATA_PATH +'/data', download=True,train = False)
    torchvision.datasets.CIFAR100(root=DATA_PATH +'/data', download=True ,train = False) 
    torchvision.datasets.SVHN(root=DATA_PATH +'/data', download=True, split = "test")


## OPTIONAL: optimization of parameters

You may use this script to test different hyperparameters
Load the network description in the file files/optim.json
The file is a json file with the following structure :
```json
{
    "optim":{ // Hyperparameters to optimize
        "model":{ // Hyperparameters to optimize for the model
            "S-LOCAL":{ // Model you want to optimize hyperparameters on 
                "hidden_size":[64,128,256,512], // List of values to test
                "dropout":[0.1,0.2,0.3,0.4,0.5] // List of values to test
            }
        }
    },
    "networks":{ // List of networks to test
        "S-LOCAL-64-0.1":{ // Name of the network
            //add the network default description here
        }
    },
    "results":{ // Results of the optimization
        "S-LOCAL-64-0.1":{ //Name of the network
            "best":{ 
                // Best hyperparameters found
            },
            "accuracy":0.9 // Accuracy of the best hyperparameters
        }
    }
}
```

In [ ]:

from time import sleep

optimer  = lambda : nnets.NetworkOptim(optim["optim"],DATA_PATH,epoch=100,verbose = True,very_verbose = False)

optim = json.load(open(INIT_PATH+"/files/optim.json"))
Thread = 4
list_to_do = [(key,item) for key,item in optim["networks"].items() if
              key not in optim["results"] and
              not (not LOCAL_TRAIN and item["model"]["name"] in ["S-LOCAL","D-LOCAL"])]
threads = [nnets.OptimThread(optim,list_to_do.pop(0),optimer()) for _ in range(Thread)]
for i in threads:
    i.start()
    sleep(1)

while len(threads) != 0:
    sleep(10)
    for i in threads:
        if i.is_alive():
            continue
        threads.remove(i)
        if len(list_to_do) == 0:
            continue
        threads.append(nnets.OptimThread(optim,list_to_do.pop(0),optimer()))
        threads[-1].start()
    

## Reproducing the paper

### Table 2


In [4]:
RETRAIN = False
table2 = pd.read_csv(INIT_PATH+"/files/table2.csv")
table2_net = json.load(open(INIT_PATH+"/files/table2.json"))
trainer = nnets.ThreadedTrainer(
    NUM_THREADS,
    DATA_PATH=".",
    SAVE_TABLE_PATH=INIT_PATH+"/files/table2.csv",
    SAVE_NETWORK_PATH= INIT_PATH+"/networks_saved/",
    SAVE_FIGURE3_PATH= INIT_PATH + "/files/figure3.json",)

for dataset in ["CIFAR-10","CIFAR-100","SVHN"]:
    for l in range(len(table2)):
        csv_line = table2.iloc[l].to_dict()
        if csv_line["train"]==False or (str(csv_line[dataset]) in "nan") or (LOCAL_TRAIN==False and csv_line["Model"] in ["S-LOCAL","D-LOCAL"]):
            continue
        ID = "_".join([csv_line[i] for i in ["Model","Training_Method"]]+[dataset])
        if not RETRAIN and os.path.isfile(INIT_PATH+"/networks_saved/"+ID+".pt"):
            continue
        network = table2_net[ID]
        trainer.add(ID,network,network["model"] == "S-FC" and network["training_method"] == "B-lasso(B=50)")
        
print("Starting training")
trainer.start()


Accuracy updated every 100 epochs
GPU available: 0.806879232GB on 8.353153024GB

     Model     |   Optimizer   |    Dataset    ||    accuracy   
---------------|---------------|---------------||----------------


Thread    ||   Thread-5  |   Thread-6  |   Thread-7  ||
----------||-------------|-------------|-------------||
Model     || S-CONV      | 3-FC        | S-FC        ||
Dataset   || CIFAR-10    | CIFAR-10    | CIFAR-10    ||
Optimizer || SGD         | SGD         | SGD         ||
cuda      || cuda        | cuda        | cuda        ||
epoch max || 4000        | 4000        | 4000        ||
Size      || 255.80 M    | 249.79 M    | 230.67 M    ||
----------||-------------|-------------|-------------||
epoch,acc || 731  66.60% | 885  59.38% | 963  59.54% ||

### figure 3
We can reuse result saved while training table 2

### figure 4